In [1]:
import subprocess

# Startet `ollama serve` im Hintergrund
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Ollama server is now running in the background.") 

Ollama server is now running in the background.


In [2]:
# Lege das LLM fest, mit dem nachfolgend gearbeitet wird.
model = "qwen2.5:7b"
model_custom = "custom_qwen2.5:7b_seed_42_temp_0"

In [3]:
# Code zum Erstellen eines Modells mit festgelegtem Seed und Temperatur in der Modelfile

# Definieren des Inhalts der Modelfile mit einem spezifischen Modell, Seed und Temperatur
seed_value = 42  # Beispiel-Seed-Wert für reproduzierbare Ergebnisse
temperature_value = 0  # Temperatur auf 0 setzen für deterministische Ausgaben

# Inhalt der Modelfile mit Seed- und Temperaturparametern
modelfile_content = f"FROM {model}\nPARAMETER seed {seed_value}\nPARAMETER temperature {temperature_value}\n"

# Schreiben des Inhalts in eine Modelfile
modelfile_path = "Modelfile"  # Pfad, um die Modelfile im aktuellen Verzeichnis zu speichern

with open(modelfile_path, "w") as modelfile:
    modelfile.write(modelfile_content)

# Befehl zum Erstellen des Modells mithilfe der angepassten Modelfile
command = ["ollama", "create", "custom_qwen2.5:7b_seed_42_temp_0", "-f", modelfile_path]

# Führen Sie den Befehl aus, um das Modell zu erstellen
try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print(result.stdout)  # Gibt die Standardausgabe aus, falls erfolgreich
except subprocess.CalledProcessError as e:
    print("An error occurred:", e.stderr)  # Zeigt eventuelle Fehlermeldungen an, falls der Befehl fehlschlägt
except FileNotFoundError:
    print("ERROR: 'ollama' command not found. Ensure that the Ollama CLI is installed and accessible.")


In [ ]:
# Gewähltes Modell von Ollama herunterladen
process = subprocess.Popen(["ollama", "pull", model], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [5]:
import faker
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
import os

# Seed setzen für Reproduzierbarkeit
random.seed(42)
Faker.seed(42)
fake = Faker()

# Ordner 'tables' erstellen, falls nicht vorhanden
output_folder = "tables"
os.makedirs(output_folder, exist_ok=True)

# Funktion, um zufällige Datumswerte zu erzeugen
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Anzahl der Datensätze, die für die Datenbank generiert werden sollen
num_students = 100
num_courses = 10
num_professors = 5
num_departments = 3
num_enrollments = 300

# 1. STUDENT_DIMENSION - Generierung der Studentendaten
student_data = {
    "Student_ID": range(1, num_students + 1),
    "First_Name": [fake.first_name() for _ in range(num_students)],
    "Last_Name": [fake.last_name() for _ in range(num_students)],
    "Date_of_Birth": [random_date(datetime(1990, 1, 1), datetime(2002, 12, 31)).date() for _ in range(num_students)],
    "Enrollment_Date": [random_date(datetime(2020, 1, 1), datetime(2023, 12, 31)).date() for _ in range(num_students)]
}

# Erstellung des DataFrames
student_df = pd.DataFrame(student_data)

# E-Mail-Adresse basierend auf First_Name und Last_Name generieren; wenn hier nochmal random verwendet wird, erhalten wir sonst abweichende Namen in der Mailadresse.
student_df["Email"] = student_df["First_Name"].str.lower() + "." + student_df["Last_Name"].str.lower() + "@example.com"

# 2. COURSE_DIMENSION - Generierung der Kursdaten
course_names = [
    "Data Science Basics", "Advanced Machine Learning", "Database Systems",
    "Statistics for Data Science", "Programming with Python", "Ethics in AI",
    "Big Data Analysis", "Data Visualization", "Project Management", "Deep Learning"
]
course_data = {
    "Course_ID": range(1, num_courses + 1),
    "Course_Name": course_names,
    "Credits": [random.choice([3, 4, 5]) for _ in range(num_courses)],
    "Department_ID": [random.randint(1, num_departments) for _ in range(num_courses)]
}
course_df = pd.DataFrame(course_data)

# 3. PROFESSOR_DIMENSION - Generierung der Professorendaten
professor_data = {
    "Professor_ID": range(1, num_professors + 1),
    "First_Name": [fake.first_name() for _ in range(num_professors)],
    "Last_Name": [fake.last_name() for _ in range(num_professors)],
    "Email": [f"{fake.first_name().lower()}.{fake.last_name().lower()}@university.com" for _ in range(num_professors)]
}
professor_df = pd.DataFrame(professor_data)

# 4. DEPARTMENT_DIMENSION - Generierung der Abteilungsdaten
department_names = ["Computer Science", "Business Administration", "Psychology"]
department_data = {
    "Department_ID": range(1, num_departments + 1),
    "Department_Name": department_names
}
department_df = pd.DataFrame(department_data)

# 5. ENROLLMENT_FACTS - Generierung der Einschreibungen
enrollment_data = {
    "Enrollment_ID": range(1, num_enrollments + 1),
    "Student_ID": [random.randint(1, num_students) for _ in range(num_enrollments)],
    "Course_ID": [random.randint(1, num_courses) for _ in range(num_enrollments)],
    "Professor_ID": [random.randint(1, num_professors) for _ in range(num_enrollments)],
    "Enrollment_Date": [random_date(datetime(2021, 1, 1), datetime(2023, 12, 31)).date() for _ in range(num_enrollments)],
    "Grade": [random.choice(['A', 'B', 'C', 'D', 'F']) for _ in range(num_enrollments)]
}
enrollment_df = pd.DataFrame(enrollment_data)

# Speichern der DataFrames als CSV-Dateien im 'tables'-Ordner
student_df.to_csv(os.path.join(output_folder, "STUDENT_DIMENSION.csv"), index=False)
course_df.to_csv(os.path.join(output_folder, "COURSE_DIMENSION.csv"), index=False)
professor_df.to_csv(os.path.join(output_folder, "PROFESSOR_DIMENSION.csv"), index=False)
department_df.to_csv(os.path.join(output_folder, "DEPARTMENT_DIMENSION.csv"), index=False)
enrollment_df.to_csv(os.path.join(output_folder, "ENROLLMENT_FACTS.csv"), index=False)

print("CSV-Dateien wurden erfolgreich im 'tables'-Ordner gespeichert.")


CSV-Dateien wurden erfolgreich im 'tables'-Ordner gespeichert.


In [6]:
import subprocess

# List available models in Ollama
process = subprocess.Popen(["ollama", "list"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ausgabe anzeigen
stdout, stderr = process.communicate()
print(stdout.decode("utf-8"))

if stderr:
    print("Error:", stderr.decode("utf-8"))
else:
    print("Available models listed successfully.")

NAME                                ID              SIZE      MODIFIED       
qwen2.5:7b                          845dbda0ea48    4.7 GB    13 seconds ago    
custom_qwen2.5:7b_seed_42_temp_0    2849624f7fb4    4.7 GB    14 seconds ago    

Available models listed successfully.


In [7]:
from src.variables import database_schema_mermaid

In [8]:
preprompt = "Your task is to generate executable SQL queries based on my provided model. Please adhere strictly to my model. For each of my questions, generate an executable SELECT SQL query. Do not provide any comments or explanations, only the query. Give me only one query per Question, make sure to start the query with 'SELECT' and end it with a ';'. I am using a sqllite-Database."

In [9]:
# Deine 5 Fachfragen in einer Liste
liste_fachfragen = [
    "Welche Kurse werden am häufigsten von Studenten wiederholt? Zeige die Kursnamen und die Anzahl der Wiederholungen.",
    "Welcher Kurs hat die meisten Teilnehmer? Zeige den Kursnamen und die Teilnehmerzahl.",
    "Wie viele Einschreibungen gab es pro Monat im letzten Jahr? Zeige die Monate und die Anzahl der Einschreibungen.",
    "Welche Studenten haben sich im letzten Jahr eingeschrieben? Zeige die Namen und Einschreibedatum.",
    "Wie viele verschiedene Fächer unterrichtet jede Abteilung? Zeige die Abteilungsnamen und Anzahl der Fächer.",
    "Welche Studenten haben Kurse aus mindestens drei verschiedenen Abteilungen belegt? Zeige die Namen der Studenten und die Anzahl der Abteilungen."
]

In [ ]:
liste_model_output = [] # Liste, in welche gleich die Ouputs des Modells gepeichert werden

# Liste mit Prompts für das Modell erstellen
prompts = [f"{database_schema_mermaid} {preprompt} {frage}" for frage in liste_fachfragen]

# Liste zum Speichern der Model Outputs
liste_model_output = []

# Schleife über die Prompts
for prompt, frage in zip(prompts, liste_fachfragen):
    # Modell für jede Frage neu starten und Prompt senden
    process = subprocess.Popen(
        ["ollama", "run", model_custom],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    # Sende den Prompt an das Modell und empfange die Antwort
    stdout, stderr = process.communicate(input=prompt.encode())
    
    # Output dekodieren und in der Liste speichern
    output = stdout.decode().strip()
    liste_model_output.append(output)  # Speichern des Outputs in der Liste

In [11]:
from src.functions import extract_sql_statement

In [12]:
# Funktion, die den Output des Modells so trimmt, dass nur das SQL-Statement stehen bleibt.
from src.functions import extract_sql_statement

# Beispielaufruf
output = stdout.decode()

# Anwenden der Extraktionsfunktion
trimmed_output = extract_sql_statement(output)


In [13]:
# Importiere die Funktion, die das SQL-Statement extrahiert
from src.functions import extract_sql_statement

# Neue Liste, die nur die getrimmten SQL-Statements speichern wird
liste_trimmed_sql = [extract_sql_statement(output) for output in liste_model_output]



In [14]:
import sqlite3
import pandas as pd

In [15]:
from src.functions import create_tables

# Datenbank und Tabellen erstellen
create_tables('university_database.db')

Tabellen wurden erfolgreich erstellt.


In [16]:
# Verbindung und Cursor erstellen
conn = sqlite3.connect('university_database.db')
cursor = conn.cursor()

In [17]:
# Zeige alle Tabellen (Namen) in der Datenbank an
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('COURSE_DIMENSION',), ('DEPARTMENT_DIMENSION',), ('ENROLLMENT_FACTS',), ('PROFESSOR_DIMENSION',), ('STUDENT_DIMENSION',)]


In [18]:
from src.functions import import_csv_to_sqlite

In [19]:
from src.functions import import_all_csv_to_sqlite

In [20]:
# Beispielaufruf für den Ordner "tables" und eine persistente Datenbank
import_all_csv_to_sqlite('tables', db_name='university_database.db')

Importiert: COURSE_DIMENSION
Tabelle 'COURSE_DIMENSION' wurde aus 'COURSE_DIMENSION.csv' importiert.
Importiert: DEPARTMENT_DIMENSION
Tabelle 'DEPARTMENT_DIMENSION' wurde aus 'DEPARTMENT_DIMENSION.csv' importiert.
Importiert: ENROLLMENT_FACTS
Tabelle 'ENROLLMENT_FACTS' wurde aus 'ENROLLMENT_FACTS.csv' importiert.
Importiert: PROFESSOR_DIMENSION
Tabelle 'PROFESSOR_DIMENSION' wurde aus 'PROFESSOR_DIMENSION.csv' importiert.
Importiert: STUDENT_DIMENSION
Tabelle 'STUDENT_DIMENSION' wurde aus 'STUDENT_DIMENSION.csv' importiert.


In [22]:
# Ausgabe des Experiments, optimiert für LaTeX

import pandas as pd

# Funktion zum Konvertieren eines DataFrames in eine LaTeX-Tabelle
def df_to_latex(df):
    return df.to_latex(index=False)

# Zähler für die Fragen initialisieren
frage_nummer = 1

# Schleife über die Fachfragen und zugehörigen SQL-Statements
for frage, sql_query in zip(liste_fachfragen, liste_trimmed_sql):
    try:
        # Führe die SQL-Abfrage aus
        cursor.execute(sql_query)
        
        # Frage und SQL-Abfrage für LaTeX-Ausgabe vorbereiten
        print(rf"\section*{{Frage {frage_nummer}:}}")
        print(frage)
        # print(r"\textbf{" + frage + "}")
        print("\n")
        
        print(r"\subsection*{Durch das Modell generierte SQL-Abfrage:}")
        print(r"\begin{minted}[fontsize=\small, breaklines, linenos]{sql}")
        print(sql_query)
        print(r"\end{minted}")
        
        # Ergebnisse der SQL-Abfrage in ein DataFrame laden und in LaTeX-Format umwandeln
        df = pd.DataFrame(cursor.fetchall(), columns=[description[0] for description in cursor.description])
        
        # DataFrame in LaTeX-Tabellenformat anzeigen
        print(r"\subsection*{Ergebnis der durch das Modell generierten SQL-Abfrage:}")
        print(df_to_latex(df))
        
        # Frage-Zähler erhöhen
        frage_nummer += 1

    except Exception as e:
        # Fehlerbehandlung bei SQL-Ausführungsfehlern in LaTeX-Format
        print(rf"\section*{{Frage {frage_nummer}: Fehler bei der Ausführung der Abfrage}}")
        print(r"\textbf{Frage:} " + frage)
        print(r"\textbf{Fehlermeldung:} " + str(e))
        print(r"\noindent\rule{\textwidth}{0.4pt}")
        

\section*{Frage 1:}
Welche Kurse werden am häufigsten von Studenten wiederholt? Zeige die Kursnamen und die Anzahl der Wiederholungen.


\subsection*{Durch das Modell generierte SQL-Abfrage:}
\begin{minted}[fontsize=\small, breaklines, linenos]{sql}
SELECT C.Course_Name, COUNT(E.Enrollment_ID) AS Repeated_Course_Count
FROM ENROLLMENT_FACTS E
JOIN COURSE_DIMENSION C ON E.Course_ID = C.Course_ID
WHERE E.Grade IN ('F', 'W')
GROUP BY C.Course_Name
ORDER BY Repeated_Course_Count DESC;
\end{minted}
\subsection*{Ergebnis der durch das Modell generierten SQL-Abfrage:}
\begin{tabular}{lr}
\toprule
                Course\_Name &  Repeated\_Course\_Count \\
\midrule
    Programming with Python &                     11 \\
              Deep Learning &                      9 \\
         Data Visualization &                      8 \\
               Ethics in AI &                      7 \\
Statistics for Data Science &                      6 \\
          Big Data Analysis &                      6 \\
